In [24]:
# Parameters
n_contributions = 21
n_punishments = 31
n_cross_val = 2
fraction_training = 0.1
data_file = "../../data/experiments/pilot_random1_player_round_slim.csv"
output_path = "../../data/training/dev"
labels = {}
model_name = "graph"
model_args = {
    "add_rnn": False,
    "add_edge_model": False,
    "add_global_model": False,
    "hidden_size": 10,
    "x_encoding": [
        {"name": "prev_contributions", "n_levels": 21, "encoding": "numeric"},
        {"name": "prev_punishments", "n_levels": 31, "encoding": "numeric"},
        {"name": "round_number", "n_levels": 16, "encoding": "numeric"},
        {"name": "prev_common_good", "norm": 128, "etype": "float"},
        {"name": "prev_valid", "etype": "bool"},
    ],
    "u_encoding": [{"name": "prev_common_good", "norm": 128, "etype": "float"}],
}
optimizer_args = {"lr": 0.001, "weight_decay": 1e-05}
train_args = {"epochs": 1000, "batch_size": 20, "clamp_grad": 1, "eval_period": 10}
shuffle_features = ['prev_punishments', 'prev_contributions', 'prev_common_good']
device = "cpu"

In [25]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import torch as th
from aimanager.generic.data import create_syn_data, create_torch_data, get_cross_validations
from aimanager.artificial_humans import AH_MODELS
from aimanager.artificial_humans.evaluation import Evaluator
from aimanager.utils.array_to_df import using_multiindex
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader

output_path = os.path.join(output_path, 'data')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
df = pd.read_csv(data_file)

data = create_torch_data(df)
syn_data = create_syn_data(n_contribution=21, n_punishment=31)

In [27]:
th_device = th.device(device)

metrics = []
confusion_matrix = []
syn_pred = []
ev = Evaluator()

th_device = th.device(device)

syn_index = ['prev_punishments', 'prev_contributions']

def create_fully_connected(n_nodes):
    return th.tensor([[i,j]
        for i in range(n_nodes)
        for j in range(n_nodes)
    ]).T

def encode(model, data, *, mask=True, index=False, x_encode=True, y_encode=True, u_encode=False, device, n_player=4):
    data = {
        'mask': data['valid'] if mask else None,
        'x': model.x_encoder(**data) if x_encode else None,
        'y_enc': model.y_encoder(**data) if y_encode else None,
        'y': data['contributions'] if y_encode else None,
        'u': model.u_encoder(**data) if u_encode and hasattr(model, 'u_encoder') else None,
        'info': th.stack([data[c] for c in syn_index], dim=-1) if index else None,
    }

    n_episodes, n_agents, n_rounds, _ = data['x'].shape

    edge_attr = th.zeros(n_player*n_player, n_rounds,0)
    edge_index = create_fully_connected(n_player)

    dataset = [
        Data(**{k: v[i] for k, v in data.items() if v is not None}, edge_attr=edge_attr, edge_index=edge_index, idx=i, group_idx=i, num_nodes=n_player).to(device)
        for i in range(n_episodes)
    ]
    return dataset

def shuffle_feature(data, feature_name):
    data = {**data}
    data[feature_name] = data[feature_name][th.randperm(len(data[feature_name]))]
    return data

for i, (train_data, test_data) in enumerate(get_cross_validations(data, n_cross_val, fraction_training)):
    model = AH_MODELS[model_name](
        n_contributions=n_contributions, n_punishments=n_punishments,
        **model_args).to(th_device)

    train_data_ = encode(model, train_data, mask=True, u_encode=True, device=th_device)
    test_data_ = encode(model, test_data, mask=True, u_encode=True, device=th_device)
    syn_data_ = encode(model, syn_data, mask=False, y_encode=False, u_encode=True, index=True, device=th_device)

    syn_df = using_multiindex(
        Batch.from_data_list(syn_data_)['info'].detach().cpu().numpy(), ['idx', 'round_number'], syn_index)

    optimizer = th.optim.Adam(model.parameters(), **optimizer_args)
    loss_fn = th.nn.CrossEntropyLoss(reduction='none')
    sum_loss = 0
    n_steps = 0

    for e in range(train_args['epochs']):
        ev.set_labels(cv_split=i, epoch=e)
        model.train()
        for j, batch_data in enumerate(iter(DataLoader(train_data_, shuffle=True, batch_size=train_args['batch_size']))):
            optimizer.zero_grad()
            py = model(batch_data).flatten(end_dim=-2)
            y_true = batch_data['y_enc'].flatten(end_dim=-2)
            mask = batch_data['mask'].flatten()
            loss = loss_fn(py, y_true)
            loss = (loss * mask).sum() / mask.sum()

            loss.backward()

            if train_args['clamp_grad']:
                for param in model.parameters():
                    param.grad.data.clamp_(-train_args['clamp_grad'], train_args['clamp_grad'])
            optimizer.step()
            sum_loss += loss.item()
            n_steps +=1
        
        last_epoch = e == (train_args['epochs'] - 1)

        if (e % train_args['eval_period'] == 0) or last_epoch:
            avg_loss = sum_loss/n_steps
            print(f'CV {i} | Epoch {e} | Loss {avg_loss}')
            ev.add_loss(avg_loss)

            ev.eval_set(model, train_data_, calc_confusion=False, set='train')
            ev.eval_set(model, test_data_, calc_confusion=last_epoch, set='test')
            for sf in shuffle_features:
                shuffled_data = shuffle_feature(test_data, sf)
                shuffled_data = encode(model, shuffled_data, mask=True, u_encode=True, device=th_device)
                ev.eval_set(model, shuffled_data, calc_confusion=False, set='test', shuffle_feature=sf)
            sum_loss = 0
            n_steps = 0
    ev.eval_syn(model, syn_data_, syn_df)

ev.save(output_path, labels)
model_path = os.path.join(output_path, 'model.pt')
model.save(model_path)

CV 0 | Epoch 0 | Loss 2.997896194458008
CV 0 | Epoch 10 | Loss 2.9791394472122192
CV 0 | Epoch 20 | Loss 2.9459847450256347
CV 0 | Epoch 30 | Loss 2.913392972946167
CV 0 | Epoch 40 | Loss 2.8804927110671996
CV 0 | Epoch 50 | Loss 2.8472017526626585
CV 0 | Epoch 60 | Loss 2.8132430791854857
CV 0 | Epoch 70 | Loss 2.7792751789093018
CV 0 | Epoch 80 | Loss 2.7462950468063356
CV 0 | Epoch 90 | Loss 2.715411114692688
CV 0 | Epoch 100 | Loss 2.6875423431396483
CV 0 | Epoch 110 | Loss 2.6635199546813966
CV 0 | Epoch 120 | Loss 2.6437491893768312
CV 0 | Epoch 130 | Loss 2.628084969520569
CV 0 | Epoch 140 | Loss 2.6160059928894044
CV 0 | Epoch 150 | Loss 2.606753873825073
CV 0 | Epoch 160 | Loss 2.5995246171951294
CV 0 | Epoch 170 | Loss 2.593610167503357
CV 0 | Epoch 180 | Loss 2.588495135307312
CV 0 | Epoch 190 | Loss 2.5838436841964723
CV 0 | Epoch 200 | Loss 2.5794502019882204
CV 0 | Epoch 210 | Loss 2.5751741886138917
CV 0 | Epoch 220 | Loss 2.570928859710693
CV 0 | Epoch 230 | Loss 2.5666